# Submission Preparation

We'll start from the JSON files submitted in the `2019-07-25-phenyl-set`:

In [1]:
!ls *.json

biphenyls_set_input.json  phenyl_set_torsiondrive_inputs.json


Our first task will be to take some of the machinery from the original submission script `03_create_torsiondrive_dataset.py` and use it here to create a QCSubmit dataset.

**Important**: Must add 1 to every initial_molecule geometry in order to trick the deduplication on the server into letting us submit these molecules.

In [2]:
import os
import json

import numpy as np

from qcsubmit.factories import TorsiondriveDatasetFactory
from openforcefield.utils.toolkits import RDKitToolkitWrapper, UndefinedStereochemistryError

In [3]:
def read_selected_torsions(input_json):
    """ Read data generated by select_torsions.py

    Returns
    -------
    selected_torsions: dict
        Dictionary for selected torsions, has this structure:
        {
            canonical_torsion_index1: {
                'initial_molecules': [ Molecule1a, Molecule1b, .. ],
                'atom_indices': [ (0,1,2,3) ],
                'attributes': {'canonical_explicit_hydrogen_smiles': .., 'canonical_isomeric_smiles': .., ..}
            },
            ..
        }
    """
    with open(input_json) as infile:
        selected_torsions = json.load(infile)
        
    return selected_torsions

First, are existing indices unique across both input files?

In [4]:
inputs = ['biphenyls_set_input.json',  'phenyl_set_torsiondrive_inputs.json']

In [5]:
set(read_selected_torsions(inputs[0]).keys()) & set(read_selected_torsions(inputs[1]).keys())

set()

Should be good! No overlap!

In [6]:
def get_qcelmol(torsion_data):
    from qcelemental.models import Molecule
    qcmol = torsion_data['input_molecules']
    
    if isinstance(qcmol, list):
        qcelmols = []
        for qcm in qcmol:
            qcm['extras'] = torsion_data['cmiles_identifiers']
            qcm['geometry'] = (np.array(qcm['geometry']) + 1.0).tolist()
            qcelmols.append(Molecule(**qcm))
    else:
        qcmol['extras'] = torsion_data["cmiles_identifiers"]
        qcmol['geometry'] = (np.array(qcmol['geometry']) + 1.0).tolist()

        qcelmols = [Molecule(**qcmol)]
        
    return qcelmols

In [7]:
factory = TorsiondriveDatasetFactory()

In [8]:
factory.scf_properties

['dipole', 'quadrupole', 'wiberg_lowdin_indices', 'mayer_indices']

In [9]:
dataset = factory.create_dataset(dataset_name="OpenFF Substituted Phenyl Set 1 v2.0",
                                 tagline="Torsiondrives for selected dihedrals of various phenyl fragments",
                                 description="Torsiondrives for selected dihedrals of various phenyl fragments",
                                 molecules=[])

In [10]:
dataset.metadata.long_description_url = "https://github.com/openforcefield/qca-dataset-submission/tree/master/2020-10-06-OpenFF-Phenyl-Set"

In [11]:
dataset.metadata.submitter = 'dotsdl'

In [12]:
def populate_dataset(dataset, input_files):
    for input_file in input_files:
        print(f"extracting from '{input_file}'")
        input_mols = read_selected_torsions(input_file)

        for key, moldata in input_mols.items():
            mol = get_qcelmol(moldata)

            #attributes = factory.create_cmiles_metadata(offmol)
            attributes = mol[0].extras
            print(f"...adding '{key}'")

            dihedral_indices = moldata['dihedral']
            if not isinstance(dihedral_indices[0], list):
                dihedral_indices = [dihedral_indices]

            if len(dihedral_indices) == 0:
                print(f"...NO DIHEDRAL INDICES FOR '{key}'")
                continue

            dihedral_indices = [list(indices) for indices in dihedral_indices]

            dataset.add_molecule(index=key,
                                 molecule=None,
                                 dihedrals=dihedral_indices,
                                 attributes=attributes,
                                 initial_molecules=mol)
    return dataset

In [13]:
ds = populate_dataset(dataset, inputs)

extracting from 'biphenyls_set_input.json'
...adding '[cH:1]1cc(cc[c:2]1[c:3]2[cH:4]c[nH+]cc2)O'
...adding '[cH:1]1cc(cc[c:2]1[c:3]2[cH:4]c[nH+]cc2)[O-]'
...adding '[cH:1]1cc(cc[c:2]1[c:3]2[cH:4]cncc2)O'
...adding '[cH:1]1cc(cc[c:2]1[c:3]2[cH:4]cncc2)[O-]'
...adding 'c1[cH:1][c:2](c(cc1O)Cl)[c:4]2[cH:3]c[nH+]cc2'
...adding 'c1[cH:1][c:2](c(cc1O)Cl)[c:4]2[cH:3]cncc2'
...adding 'c1[cH:1][c:2](c(cc1O)F)[c:4]2[cH:3]c[nH+]cc2'
...adding 'c1[cH:1][c:2](c(cc1O)F)[c:4]2[cH:3]cncc2'
...adding 'c1[cH:1][c:2](c(cc1[O-])Cl)[c:4]2[cH:3]c[nH+]cc2'
...adding 'c1[cH:1][c:2](c(cc1[O-])Cl)[c:4]2[cH:3]cncc2'
...adding 'c1[cH:1][c:2](c(cc1[O-])F)[c:4]2[cH:3]c[nH+]cc2'
...adding 'c1[cH:1][c:2](c(cc1[O-])F)[c:4]2[cH:3]cncc2'
...adding 'c1c[cH:1][c:2](cc1)[c:3]2[cH:4]cncc2'
...adding 'c1ccc([c:2]([cH:1]1)[c:3]2[cH:4]cncc2)Cl'
...adding 'c1ccc([c:2]([cH:1]1)[c:3]2[cH:4]cncc2)F'
extracting from 'phenyl_set_torsiondrive_inputs.json'
...adding 'CCC(=O)Nc1[cH:1][c:2](ccn1)[NH:3][CH2:4]C'
...adding 'CCCNc1cc[c:2](

/home/david/.conda/envs/qcsubmit/lib/python3.7/site-packages/qcsubmit/validators.py:181: UserWarning: The molecule C11H10NO has a net charge of 1.0.
  UserWarning,
/home/david/.conda/envs/qcsubmit/lib/python3.7/site-packages/qcsubmit/validators.py:181: UserWarning: The molecule C11H8NO has a net charge of -1.0.
  UserWarning,
/home/david/.conda/envs/qcsubmit/lib/python3.7/site-packages/qcsubmit/validators.py:181: UserWarning: The molecule C11ClH9NO has a net charge of 1.0.
  UserWarning,
/home/david/.conda/envs/qcsubmit/lib/python3.7/site-packages/qcsubmit/validators.py:181: UserWarning: The molecule C11FH9NO has a net charge of 1.0.
  UserWarning,
/home/david/.conda/envs/qcsubmit/lib/python3.7/site-packages/qcsubmit/validators.py:181: UserWarning: The molecule C11ClH7NO has a net charge of -1.0.
  UserWarning,
/home/david/.conda/envs/qcsubmit/lib/python3.7/site-packages/qcsubmit/validators.py:181: UserWarning: The molecule C11FH7NO has a net charge of -1.0.
  UserWarning,


...adding 'CCCNc1ccc[c:2]([cH:1]1)[N:3]([CH3:4])C'
...adding 'CCCNc1ccc[c:2]([cH:1]1)[NH:3][CH3:4]'
...adding 'CCNc1[cH:1][c:2](ccn1)[NH:3][C:4](=O)NC'
...adding 'CCNc1[cH:1][c:2](ccn1)[O:3][C:4](=O)N'
...adding 'CCOC(=O)c1cc[cH:1][c:2](c1)[N+:3](=O)[O-:4]'
...adding 'CCOC(=O)c1ccc[c:2]([cH:1]1)[NH:3][C:4](=O)N(C)C'
...adding 'CCO[C:3](=[O:4])[c:2]1[cH:1]cc(cc1)NC(=O)C'
...adding 'CCO[C:3](=[O:4])[c:2]1[cH:1]cnc(c1)[N+](C)(C)C'
...adding 'CCOc1[cH:1][c:2](ccn1)[NH:3][C:4](=O)NC'
...adding 'CCOc1[cH:1][c:2](cnc1)[O:3][CH3:4]'
...adding 'CCOc1cc[c:2]([cH:1]n1)[C:3](=O)[O:4]CC'
...adding 'CC[C:4](=O)[NH:3][c:2]1[cH:1]c(cnc1)N'
...adding 'CC[C:4](=O)[NH:3][c:2]1[cH:1]cc(cc1)F'
...adding 'CC[C:4](=O)[NH:3][c:2]1cc(cn[cH:1]1)[N+](C)(C)C'
...adding 'CC[C:4](=O)[NH:3][c:2]1ccc(c[cH:1]1)N(C)C'
...adding 'CC[C:4](=O)[NH:3][c:2]1ccc(n[cH:1]1)C(=O)OCC'
...adding 'CC[C:4](=O)[NH:3][c:2]1ccc(n[cH:1]1)[N+](=O)[O-]'
...adding 'CC[C:4](=O)[NH:3][c:2]1ccc(n[cH:1]1)[N+](C)(C)C'
...adding 'CC[C:4](=O)[NH:

/home/david/.conda/envs/qcsubmit/lib/python3.7/site-packages/qcsubmit/validators.py:181: UserWarning: The molecule C11H17N2O2 has a net charge of 1.0.
  UserWarning,
/home/david/.conda/envs/qcsubmit/lib/python3.7/site-packages/qcsubmit/validators.py:181: UserWarning: The molecule C11H18N3O has a net charge of 1.0.
  UserWarning,


...adding 'CC[CH2:4][NH:3][c:2]1[cH:1]cc(nc1)NC(=O)NC'
...adding 'CC[CH2:4][NH:3][c:2]1cc(cn[cH:1]1)C(=O)OCC'
...adding 'CC[CH2:4][NH:3][c:2]1ccc(c[cH:1]1)[N+](C)(C)C'
...adding 'CC[CH2:4][NH:3][c:2]1ccc(n[cH:1]1)[N+](C)(C)C'
...adding 'CC[CH2:4][NH:3][c:2]1cccc([cH:1]1)NC(=O)N(C)C'


...

...adding 'C[N+](C)(C)c1ccc[c:2]([cH:1]1)[NH:3][C:4](=O)N'
...adding 'C[N:3]([CH3:4])[c:2]1[cH:1]ccc(c1)NC(=O)N(C)C'
...adding 'C[N:3]([CH3:4])[c:2]1cc(cn[cH:1]1)[N+](C)(C)C'
...adding 'C[N:3]([CH3:4])[c:2]1ccc(c[cH:1]1)[N+](C)(C)C'
...adding '[CH3:4][N:3](C)[c:2]1[cH:1]cc(cc1)N(C)C'
...adding '[CH3:4][N:3](C)[c:2]1[cH:1]cc(nc1)[N+](=O)[O-]'
...adding '[CH3:4][N:3](C)[c:2]1[cH:1]cc(nc1)[N+](C)(C)C'
...adding '[CH3:4][N:3](C)[c:2]1[cH:1]ccc(c1)OC(=O)N'
...adding '[CH3:4][N:3](C)[c:2]1ccc(n[cH:1]1)C(F)(F)F'
...adding '[CH3:4][NH:3][c:2]1[cH:1]c(cnc1)C(=O)O'
...adding '[CH3:4][NH:3][c:2]1[cH:1]cc(cc1)NC'
...adding '[CH3:4][NH:3][c:2]1[cH:1]cc(cc1)[N+](=O)[O-]'
...adding '[CH3:4][NH:3][c:2]1[cH:1]cc(nc1)C(=O)O'
...adding '[CH3:4][NH:3][c:2]1[cH:1]cc(nc1)O'
...adding '[CH3:4][NH:3][c:2]1[cH:1]cc(nc1)[N+](C)(C)C'
...adding '[CH3:4][NH:3][c:2]1[cH:1]cnc(c1)N'


/home/david/.conda/envs/qcsubmit/lib/python3.7/site-packages/qcsubmit/validators.py:181: UserWarning: The molecule C9H16N3 has a net charge of 1.0.
  UserWarning,
/home/david/.conda/envs/qcsubmit/lib/python3.7/site-packages/qcsubmit/validators.py:181: UserWarning: The molecule C8H14N3 has a net charge of 1.0.
  UserWarning,
/home/david/.conda/envs/qcsubmit/lib/python3.7/site-packages/qcsubmit/validators.py:181: UserWarning: The molecule C9H15N2 has a net charge of 1.0.
  UserWarning,
/home/david/.conda/envs/qcsubmit/lib/python3.7/site-packages/qcsubmit/validators.py:181: UserWarning: The molecule C8H13N2O has a net charge of 1.0.
  UserWarning,


...adding '[CH3:4][NH:3][c:2]1cc(cn[cH:1]1)[N+](C)(C)C'
...adding '[CH3:4][NH:3][c:2]1ccnc([cH:1]1)[N+](C)(C)C'
...adding '[CH3:4][O:3][c:2]1[cH:1]cc(nc1)[N+](=O)[O-]'
...adding '[CH3:4][O:3][c:2]1[cH:1]ccc(c1)C(=O)O'
...adding '[CH3:4][O:3][c:2]1[cH:1]cnc(c1)I'
...adding '[H:4][NH:3][c:2]1[cH:1]c(cnc1)NC(=O)NC'
...adding '[H:4][NH:3][c:2]1[cH:1]c(cnc1)[N+](C)(C)C'
...adding '[H:4][NH:3][c:2]1[cH:1]cc(cc1)C(=O)OCC'
...adding '[H:4][NH:3][c:2]1[cH:1]cc(cc1)N(C)C'
...adding '[H:4][NH:3][c:2]1[cH:1]cc(cc1)[N+](C)(C)C'
...adding '[H:4][NH:3][c:2]1[cH:1]cc(nc1)OC'
...adding '[H:4][NH:3][c:2]1[cH:1]cc(nc1)[N+](=O)[O-]'
...adding '[H:4][NH:3][c:2]1[cH:1]cnc(c1)[N+](=O)[O-]'
...adding '[H:4][NH:3][c:2]1ccc(n[cH:1]1)[N+](C)(C)C'
...adding '[H:4][O:3][c:2]1[cH:1]cc(cc1)[N+](=O)[O-]'
...adding '[H:4][O:3][c:2]1[cH:1]ccc(c1)C'
...adding '[H:4][O:3][c:2]1[cH:1]cnc(c1)N'
...adding '[H:4][O:3][c:2]1cc(cn[cH:1]1)NC(=O)N(C)C'
...adding '[H:4][O:3][c:2]1cc(cn[cH:1]1)[N+](C)(C)C'
...adding '[H:4][O:3][c:

/home/david/.conda/envs/qcsubmit/lib/python3.7/site-packages/qcsubmit/validators.py:181: UserWarning: The molecule C9H14NO has a net charge of 1.0.
  UserWarning,


...adding '[cH:1]1[c:2](cncc1OC(=O)N)[NH:3][C:4](=O)N'
...adding '[cH:1]1cc(nc[c:2]1[C:3](=[O:4])O)C#N'
...adding '[cH:1]1cc(nc[c:2]1[N+:3](=O)[O-:4])F'
...adding '[cH:1]1cc(nc[c:2]1[N+:3](=O)[O-:4])NC(=O)N'
...adding '[cH:1]1cc(nc[c:2]1[NH:3][C:4](=O)N)[N+](=O)[O-]'
...adding '[cH:1]1cc(nc[c:2]1[O:3][C:4](=O)N)[N+](=O)[O-]'
...adding '[cH:1]1cnc(c[c:2]1[N+:3](=O)[O-:4])N'
...adding '[cH:1]1cnc(c[c:2]1[NH:3][C:4](=O)N)[N+](=O)[O-]'
...adding 'c1cc(c[c:2]([cH:1]1)[NH:3][C:4](=O)N)C(=O)O'
...adding 'c1cc(c[cH:1][c:2]1[C:3](=[O:4])O)NC(=O)N'
...adding 'c1cnc([cH:1][c:2]1[C:3](=[O:4])O)NC(=O)N'


This dataset also had iodine cases that won't converge with the current SCF iteration number.
We could try upping the SCF iteration number.

In [14]:
# want to test this out on the iodine cases to ensure convergence
dataset.maxiter = 400

Try doubling, run on an iodine case, see if we get convergence.

In [15]:
dataset.scf_properties

['dipole', 'quadrupole', 'wiberg_lowdin_indices', 'mayer_indices']

In [18]:
dataset.export_dataset('dataset.json')

In [20]:
dataset.visualize('molecules.pdf')

## Testing convergence of iodine cases

In [39]:
mol_iodine = list()
for mol in dataset.molecules:
    if 'I' in [atom.element.symbol for atom in mol.atoms]:
        mol_iodine.append(mol)

In [40]:
mol_iodine

[Molecule with name 'CC[CH2:4][NH:3][c:2]1ccnc([cH:1]1)I' and SMILES '[H]c1c(nc(c(c1N([H])C([H])([H])C([H])([H])C([H])([H])[H])[H])I)[H]',
 Molecule with name 'CN(C)[C:4](=O)[NH:3][c:2]1[cH:1]cc(cc1)I' and SMILES '[H]c1c(c(c(c(c1N([H])C(=O)N(C([H])([H])[H])C([H])([H])[H])[H])[H])I)[H]',
 Molecule with name 'C[C:4](=O)[NH:3][c:2]1[cH:1]c(cnc1)I' and SMILES '[H]c1c(c(nc(c1I)[H])[H])N([H])C(=O)C([H])([H])[H]',
 Molecule with name 'C[C:4](=O)[NH:3][c:2]1[cH:1]cnc(c1)I' and SMILES '[H]c1c(nc(c(c1N([H])C(=O)C([H])([H])[H])[H])I)[H]',
 Molecule with name 'C[CH2:4][O:3][c:2]1[cH:1]cc(cc1)I' and SMILES '[H]c1c(c(c(c(c1OC([H])([H])C([H])([H])[H])[H])[H])I)[H]',
 Molecule with name '[CH3:4][O:3][c:2]1[cH:1]cnc(c1)I' and SMILES '[H]c1c(nc(c(c1OC([H])([H])[H])[H])I)[H]']

In [42]:
type(mol_iodine[0])

openforcefield.topology.molecule.Molecule

Create a dataset with just these, submit to local QCArchive for test.

Instead of filtering here, just use the same functions from above, but only take molecules with iodine.